In [1]:
import numpy as np
import pandas as pd
import sima2py as sapy

/home/phys/villaa/python/anaconda3/envs/py365/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/phys/villaa/python/anaconda3/envs/py365/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#this is great, but I need to be able to get more events, that means analyzing/skimming multiple files
import re

#fileregex = re.compile('Run68_gdirect_bknd_R68_PuBe_0x0006_1M_155076\\S+.txt')
#dirpath,f = sapy.listFiles('/data/chocula/villaa/k100Sim_Data/captureCal/',fileregex)

In [3]:
#print(np.size(f))

# Full Skim of this Data (w/ and w/o Captures)

I've now made large skims of the data with approximately 22 hrs of livetime assuming 1M n/s from the Pu/Be source. 

 

In [4]:
#play around with some hits data stored in h5 file
#===============to suppress h5py warning see:
#https://github.com/h5py/h5py/issues/961
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()

f_nocap = h5py.File("/data/chocula/villaa/k100Sim_Data/captureCalhdf5/R68_gdirect_testskim_megahighstat_nocap_new.h5","r")
    
data_nocap = f_nocap['geant4/hits']

f_nocap_er = h5py.File("/data/chocula/villaa/k100Sim_Data/captureCalhdf5/R68_gdirect_testskim_megahighstat_nocap_er_lowe_new.h5","r")
    
data_nocap_er = f_nocap_er['geant4/hits']

print(np.shape(data_nocap))
print(np.shape(data_nocap_er))

(211370, 22)
(51420, 22)


In [ ]:
#now make a dataframe with the restricted data
nr_data_nocap = data_nocap[:,[0,4,5,6,21]]
nr_dataframe_nocap = pd.DataFrame(data=nr_data_nocap)
er_data_nocap = data_nocap_er[:,[0,4,5,6,21]]
er_dataframe_nocap = pd.DataFrame(data=er_data_nocap)

#print(nr_dataframe)

groupbyvec=[0]
#print(np.max(nr_dataframe.groupby([0,1],axis=0).size()))
    
max_vec_nocap = np.max(nr_dataframe_nocap.groupby(groupbyvec,axis=0).size())

evec_nocap = np.zeros((0,max_vec_nocap))
nhit_nocap = np.zeros((0,1))

for i in nr_dataframe_nocap.groupby(groupbyvec,axis=0)[3].apply(list):
    #print(i)
    #print(np.shape(i))
    vector = np.zeros((1,max_vec_nocap))
    #print(np.shape(vector[0,0:np.shape(i)[0]]))
    vector[0,0:np.shape(i)[0]] = np.transpose(np.asarray(i))
    evec_nocap = np.append(evec_nocap,vector*1e6,0) #convert from MeV
    nhit_nocap = np.append(nhit_nocap,np.shape(i)[0])
    
max_vec_nocap_er = np.max(er_dataframe_nocap.groupby(groupbyvec,axis=0).size())

evec_nocap_er = np.zeros((0,max_vec_nocap_er))
nhit_nocap_er = np.zeros((0,1))

print(np.shape(er_dataframe_nocap))
print(np.shape(nr_dataframe_nocap))
for n,i in enumerate(er_dataframe_nocap.groupby(groupbyvec,axis=0)[3].apply(list)):
    #if n>36000: break
    #print(i)
    #print('{} out of {}'.format(n,np.shape(er_dataframe_nocap)[0]))
    #print(np.shape(i))
    vector = np.zeros((1,max_vec_nocap_er))
    #print(np.shape(vector[0,0:np.shape(i)[0]]))
    vector[0,0:np.shape(i)[0]] = np.transpose(np.asarray(i))
    #print(np.shape(evec_nocap_er))
    #print(np.shape(vector))
    #print(vector)
    evec_nocap_er = np.append(evec_nocap_er,vector*1e6,0) #convert from MeV
    nhit_nocap_er = np.append(nhit_nocap_er,np.shape(i)[0])

In [ ]:
import damic_y as dy 

#a spline extrapolation to DAMIC data
damic_y = dy.getDAMICy()
damic_yv = np.vectorize(damic_y) #vectorize it

In [ ]:
print(damic_y(2000))

In [ ]:
#convert evec to eVee
V = 120
eps = 3.8 #eV for silicon
F = 0.1161 #silicon value taken from https://www.sciencedirect.com/science/article/pii/S0168900297009650
#evec_ee = (evec + evec*damic_y(evec)*(V/eps))/(1+(V/eps))
evec_ee_nocap = (evec_nocap + evec_nocap*damic_y(evec_nocap)*(V/eps))/(1+(V/eps))
evec_ee_nocap_er = (evec_nocap_er + evec_nocap_er*(V/eps))/(1+(V/eps))

In [ ]:
#make some histograms

xmax = 1000
thresh = 0 #at best a 7 eV threshold
#sume = np.sum(evec_ee,1)
#n,nx = np.histogram(sume[sume>thresh],100,range=(0,xmax))
sume_nocap = np.sum(evec_ee_nocap,1)
n_nocap,nx_nocap = np.histogram(sume_nocap[sume_nocap>thresh],100,range=(0,xmax))
sume_nocap_er = np.sum(evec_ee_nocap_er,1)
n_nocap_er,nx_nocap_er = np.histogram(sume_nocap_er[sume_nocap_er>thresh],100,range=(0,xmax))
#n_ss,nx_ss = np.histogram(np.sum(nr_energies[nr_hits==1,:],1)*1000,250,range=(0,xmax))
#n_ms,nx_ms = np.histogram(np.sum(nr_energies[nr_hits>1,:],1)*1000,250,range=(0,xmax))




xc = (nx_nocap[:-1] + nx_nocap[1:]) / 2

In [ ]:
#set up a plot 
import matplotlib as mpl
from matplotlib import pyplot as plt
#got smarter about the mpl config: see mplstyles/ directory
plt.style.use('standard')

In [ ]:
#fpube = open('data/r68_n125_PuBe_PTOFkeV_1keV_spikesremoved_scan_fmt.txt')
fpube = open('data/r68_n125_PuBe_cgood_cmid_final_PTOFkeV_2keV_scan_fmt.txt') #newest version of data
fbknd = open('data/r68_n125_bkg_PTOFkeV_1keV_spikesremoved_scan_fmt.txt')
#fpube = open('data/r68_n125_PuBe_cgood_PTOFkeV0_1keV_scan_fmt.txt')
#fbknd = open('data/r68_n125_Bkg_cgood_PTOFkeV0_1keV_scan_fmt.txt')

d = np.asarray([x.split() for x in fpube.readlines()],dtype=np.float)
db = np.asarray([x.split() for x in fbknd.readlines()],dtype=np.float)

dE = d[:,0]
dbE = db[:,0]
print(np.shape(dE))
print(np.shape(dbE))

In [ ]:
#want to include Nick's measured trigger efficiency (email on 4/11/2019)
import scipy.special as special
Offset = 0.636
E0 = 26.97
sigma = 13.77

trigeff = lambda x: Offset+(1-Offset)*0.5*(1+special.erf((x-E0)/(2**0.5*sigma)))
trigeffv = np.vectorize(trigeff)


In [ ]:
#get histogram
#emax = 500 #emax in eV
emax=1000
nbins = 100

nd_68,nxd_68 = np.histogram(dE*1000,nbins,range=(0,emax))

xcd_68 = (nxd_68[:-1] + nxd_68[1:]) / 2

ndb_68,nxdb_68 = np.histogram(dbE*1000,nbins,range=(0,emax))

xcdb_68 = (nxdb_68[:-1] + nxdb_68[1:]) / 2



In [ ]:
#set up a 1d plot
fig,axes = plt.subplots(1,1,figsize=(9.0,8.0),sharex=True)
ax1 = axes

livefac = (39.1/19.6)*(1.96/1) #livetime fraction estimated from N. Mast see email from March 28 2019.
#he estimates 39.1 live hours of data
#my simulation corresponds to 22 live hours if 1e6 neutrons/sec
#or 11 live hours if 2e6 neutrons/second Pu/Be source. 
#actually the best estimate for source neutrons is 1.96e6 events/sec, this is where the factor above came from. 

#print(xc)
#print(trigeffv(xc))
ymin=10
ymax=10000
#ax1.step(xc,n*livefac, where='mid',color='k', linestyle='-', \
#         label='all scatters', linewidth=2)
ax1.step(xc,trigeffv(xc)*n_nocap*livefac, where='mid',color='royalblue', linestyle='-', \
         label='sim. w/o captures (NR only)', linewidth=2)
ax1.step(xc,trigeffv(xc)*n_nocap_er*livefac, where='mid',color='orange', linestyle='-', \
         label='sim. w/o captures (ER only)', linewidth=2)
ax1.step(xc,trigeffv(xc)*(n_nocap+n_nocap_er)*livefac, where='mid',color='r', linestyle='-', \
         label='sim. w/o captures (ERs and NRs)', linewidth=2)

dlabel='data minus background'
#step1, = ax1.step(xcd_68,nd_68-ndb_68, where='mid',color='k', linestyle='-', label=dlabel, linewidth=2)
err = np.sqrt(nd_68+ndb_68)
ax1.errorbar(xcd_68,nd_68-ndb_68,yerr=[err,err], marker='o', markersize=6, \
             ecolor='k',color='k', linestyle='none', label=dlabel, linewidth=2)




#tlabel = 'Thresh. {0} eV$_{{\mathrm{{ee}}}}$'.format(18)
#ax1.axvline(thresh, color='k', linestyle='--', lw=2, alpha=0.8,label=tlabel)
#erange_x = np.arange(thresh-sigthr, thresh+sigthr, 0.01)
#ax1.fill_between(erange_x, ymin, ymax, facecolor='r', alpha=0.3)

ax1.set_yscale('log')
ax1.set_xlim(0, xmax) #in pairs
ax1.set_ylim(ymin,ymax)
ax1.set_xlabel('total deposited energy [eV$_{\\mathrm{ee}}$]',**axis_font)
ax1.set_ylabel('counts',**axis_font)
ax1.grid(True)
ax1.yaxis.grid(True,which='minor',linestyle='--')
ax1.legend(loc=1,prop={'size':22})

for axis in ['top','bottom','left','right']:
  ax1.spines[axis].set_linewidth(2)

plt.tight_layout()
plt.savefig('figures/R68_sim_gdirect_bknd_compare_190412.png')
plt.show()

In [ ]:
#make some histograms

xmax = 2000
thresh = 7 #at best a 7 eV threshold
#sume = np.sum(evec_ee,1)
#n,nx = np.histogram(sume[sume>thresh],100,range=(0,xmax))
sume_nocap = np.sum(evec_ee_nocap,1)
n_nocap,nx_nocap = np.histogram(sume_nocap[sume_nocap>thresh],100,range=(0,xmax))
sume_nocap_er = np.sum(evec_ee_nocap_er,1)
n_nocap_er,nx_nocap_er = np.histogram(sume_nocap_er[sume_nocap_er>thresh],100,range=(0,xmax))
#n_ss,nx_ss = np.histogram(np.sum(nr_energies[nr_hits==1,:],1)*1000,250,range=(0,xmax))
#n_ms,nx_ms = np.histogram(np.sum(nr_energies[nr_hits>1,:],1)*1000,250,range=(0,xmax))




xc = (nx_nocap[:-1] + nx_nocap[1:]) / 2

In [ ]:
#get histogram
#emax = 500 #emax in eV
emax=2000
nbins = 100

nd_68,nxd_68 = np.histogram(dE*1000,nbins,range=(0,emax))

xcd_68 = (nxd_68[:-1] + nxd_68[1:]) / 2

ndb_68,nxdb_68 = np.histogram(dbE*1000,nbins,range=(0,emax))

xcdb_68 = (nxdb_68[:-1] + nxdb_68[1:]) / 2

In [ ]:
#set up a 1d plot
fig,axes = plt.subplots(1,1,figsize=(9.0,8.0),sharex=True)
ax1 = axes

livefac = (39.1/19.6)*(1.96/1) #livetime fraction estimated from N. Mast see email from March 28 2019.
#he estimates 39.1 live hours of data
#my simulation corresponds to 22 live hours if 1e6 neutrons/sec
#or 11 live hours if 2e6 neutrons/second Pu/Be source. 
#actually the best estimate for source neutrons is 1.96e6 events/sec, this is where the factor above came from. 

ymin=10
ymax=100000
#ax1.step(xc,n*livefac, where='mid',color='k', linestyle='-', \
#         label='all scatters', linewidth=2)
ax1.step(xc,n_nocap*livefac, where='mid',color='royalblue', linestyle='-', \
         label='sim. w/o captures (NR only)', linewidth=2)
ax1.step(xc,n_nocap_er*livefac, where='mid',color='orange', linestyle='-', \
         label='sim. w/o captures (ER only)', linewidth=2)
ax1.step(xc,(n_nocap+n_nocap_er)*livefac, where='mid',color='r', linestyle='-', \
         label='sim. w/o captures (ERs and NRs)', linewidth=2)

dlabel='data minus background'
#step1, = ax1.step(xcd_68,nd_68-ndb_68, where='mid',color='k', linestyle='-', label=dlabel, linewidth=2)
err = np.sqrt(nd_68+ndb_68)
ax1.errorbar(xcd_68,nd_68-ndb_68,yerr=[err,err], marker='o', markersize=6, \
             ecolor='k',color='k', linestyle='none', label=dlabel, linewidth=2)




#tlabel = 'Thresh. {0} eV$_{{\mathrm{{ee}}}}$'.format(18)
#ax1.axvline(thresh, color='k', linestyle='--', lw=2, alpha=0.8,label=tlabel)
#erange_x = np.arange(thresh-sigthr, thresh+sigthr, 0.01)
#ax1.fill_between(erange_x, ymin, ymax, facecolor='r', alpha=0.3)

ax1.set_yscale('log')
ax1.set_xlim(0, xmax) #in pairs
ax1.set_ylim(ymin,ymax)
ax1.set_xlabel('total deposited energy [eV$_{\\mathrm{ee}}$]',**axis_font)
ax1.set_ylabel('counts',**axis_font)
ax1.grid(True)
ax1.yaxis.grid(True,which='minor',linestyle='--')
#ax1.legend(loc=1,prop={'size':22})

for axis in ['top','bottom','left','right']:
  ax1.spines[axis].set_linewidth(2)

plt.tight_layout()
plt.savefig('figures/R68_sim_gdirect_bknd_compare_2keV_190412.png')
plt.show()